# Modification of database to run scenarios

Here we modify the technology matrix without touching the files in the database, this is useful to e.g. modify bacgkround databases like ecoinvent to simulate scenarios

1. Import BW2 
2. create a new project 'biowood'
3. import biosphere3
4. import ecoinvent database (v.3.7.1 conseq)
5. modify specific activities in the ecoinvent database (using the technology matrix coordinates)
6. calculate results with the modified ecoinvent database

You will need ecoinvent v.7 for this or you can change the activity codes if you want to use other versions

**Set up**

In [1]:
import random
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt
import matplotlib as mpl


In [2]:
import brightway2 as bw

In [3]:
bw.projects # check what project you have 
# bw.projects.delete_project('advlca22', delete_dir=True) # if you want a fresh start

Brightway2 projects manager with 36 objects, including:
	B4B18
	BTC02
	BTC_Att_LCA
	Biowood
	C3BO
	CCU
	ConseqUncertainty
	GSA
	Giovanni
	HH
Use `sorted(projects)` to get full list, `projects.report()` to get
	a report on all projects.

In [28]:
# bw.databases.clear()
# bw.methods.clear()

In [4]:
bw.projects.set_current('Biowood') # Still working in the same project
bw.databases
#bw.databases.clear() # For a fresh start (Risky command! clears all your existing databases)

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.7.1 conseq_fs
	fg_bw

## Run the code below only the first time

Before importing ecoinvent, we need to make a default setup of Brightway2. This means importing all the environmental exchanges and all the LCIA methods. Then when we import ecoinvent the ecoinvent activities will be linked to this database of environmental exchanges, just like in the example wiht the product system of Hejungs and Suh (2002).

In [4]:
# Import the biosphere3 database
bw.bw2setup()  # This will take some time

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/02/2021 13:01:22
  Finished: 12/02/2021 13:01:23
  Total time elapsed: 00:00:00
  CPU %: 81.10
  Memory %: 1.12
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 2.00 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



In [5]:
bw.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.7.1 conseq_fs
	fg_bw

**We are going to use version 3.7.1 of ecoinvent, consequential model.**

In [5]:
# Import ecoinvent , /Users/staff/Desktop/Biowood/LCA/CLCA BR2 model/Model 1/Ecoinvent/ecoinvent 3.6_consequential_ecoSpold02/datasets

# You need to change the line below with the directory where you have saved ecoinvent
ei37fsdir = "/Users/massimo/Documents/Databases/ecoinvent v3.7.1/ecoinvent 3.7.1_consequential_ecoSpold02/datasets"

if 'ecoinvent 3.7.1 conseq_fs' in bw.databases:
    print("Database has already been imported")
else:
    ei37fs = bw.SingleOutputEcospold2Importer(ei37fsdir, 'ecoinvent 3.7.1 conseq_fs', use_mp = False) # You can give it another name of course
    ei37fs.apply_strategies()
    ei37fs.statistics()

ei37fs.drop_unlinked(True)
ei37fs.write_database() # This will take some time.

Extracting ecospold2 files:
0% [##############################] 100% | ETA: 00:00:00 | Item ID: 3a7fd0fe-3f4a-4
Total time elapsed: 00:04:26


Title: Extracting ecospold2 files:
  Started: 12/02/2021 13:05:14
  Finished: 12/02/2021 13:09:41
  Total time elapsed: 00:04:26
  CPU %: 88.50
  Memory %: 16.09
Extracted 17328 datasets in 267.27 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
15 exchanges couldn't be lin

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:53


Title: Writing activities to SQLite3 database:
  Started: 12/02/2021 13:09:53
  Finished: 12/02/2021 13:10:46
  Total time elapsed: 00:00:53
  CPU %: 87.50
  Memory %: 16.39
Created database: ecoinvent 3.7.1 conseq_fs


Brightway2 SQLiteBackend: ecoinvent 3.7.1 conseq_fs

In [6]:
bw.databases # you should now see both "biosphere3" and "ecoinvent 3.7.1 conseq"

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.7.1 conseq_fs
	fg_bw

## Start here if you have already imported the background database

Create a simple foreground process that uses 1 kWh electricity in Belgium

In [37]:
fg_bw = bw.Database("fg_bw") # foreground biowood
fg_bw.write({
    ("fg_bw", "biowood production"):{ # Note that a tuple is used to identify an activity univocally
        'name':'biowood production',
        'unit': 'kilogram', 
        'exchanges': [{
                'input': ('ecoinvent 3.7.1 conseq_fs', 'e5bf639cfd7f62017f0f72da8e0dc5f4'), # 'market for electricity, high voltage' (kilowatt hour, BE, None)
                'amount': 1,
                'unit': 'kilowatt hour',
                'type': 'technosphere'
            }]}})
    

Writing activities to SQLite3 database:
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/11/2022 21:11:24
  Finished: 04/11/2022 21:11:24
  Total time elapsed: 00:00:00
  CPU %: 59.10
  Memory %: 2.28


In [63]:
#you can use this doe to check the inputs of the belgian mix
belgian_mix = bw.Database('ecoinvent 3.7.1 conseq_fs').get('e5bf639cfd7f62017f0f72da8e0dc5f4') 
for exc in belgian_mix.exchanges():
    print(exc['type'])
    print(exc['input'][1])
    print(exc.input)
    print(exc['amount'])
    print("-------")

technosphere
1d9ae44baaa315ca0589601ffd7121f7
'transmission network construction, long-distance' (kilometer, UCTE, None)
3.17e-10
-------
technosphere
a0f8a6082de3c35932ac46a5cbccf340
'market for transmission network, electricity, high voltage' (kilometer, GLO, None)
6.58209848825026e-09
-------
production
e5bf639cfd7f62017f0f72da8e0dc5f4
'market for electricity, high voltage' (kilowatt hour, BE, None)
1.0
-------
technosphere
1914a838e222c50aa695ffd1999cb9b5
'electricity production, hydro, run-of-river' (kilowatt hour, BE, None)
0.00602776051608669
-------
technosphere
36f4649326c821b4e8dc8e005f83efd3
'electricity production, natural gas, combined cycle power plant' (kilowatt hour, BE, None)
0.557342534494605
-------
technosphere
8928815498415253f89532b9a6233ae0
'electricity production, oil' (kilowatt hour, BE, None)
0.0180105591347089
-------
technosphere
89578c816af5949efef19da4d5015ecc
'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, BE, None)
0.2009371900102

# Replace/change the activity

Find the products and activities that you need.

In particular we want to chenge the shared of wind and gas in the belgian electircity mix

In [38]:
products_to_change = [('ecoinvent 3.7.1 conseq_fs', '89578c816af5949efef19da4d5015ecc'),  # wind
                       ('ecoinvent 3.7.1 conseq_fs', '36f4649326c821b4e8dc8e005f83efd3')]  # gas

# wind == 'electricity production, wind, 1-3MW turbine, offshore' (kilowatt hour, BE, None)
# gas == 'electricity production, natural gas, combined cycle power plant' (kilowatt hour, BE, None)

In [39]:
activities_to_change = [('ecoinvent 3.7.1 conseq_fs', 'e5bf639cfd7f62017f0f72da8e0dc5f4')]  # belgian el mix

Initialize the LCA using the FU (this could be the BAU scenario)

In [40]:
LCA = bw.LCA({("fg_bw", "biowood production") : 1}, 
             ('IPCC 2013', 'climate change', 'GWP 100a'))
LCA.lci()
LCA.lcia()

In [41]:
LCA.score

0.2632287072404267

Decide which activities and exchanges to modify. 

_(You can dedice yourself hot to structure this, below is only a suggestion)_

I use here a list of **tuples** each tuple with this structure: (activity, exchange, new value) 
- first element is the column in tech matrix, 
- second is the row in the tech matrix, 
- third element of the tuple is the new value (scenario)

So the first two elements are coordinates and the third is a value

In [49]:
to_change = [(('ecoinvent 3.7.1 conseq_fs', 'e5bf639cfd7f62017f0f72da8e0dc5f4'), 
             ('ecoinvent 3.7.1 conseq_fs', '89578c816af5949efef19da4d5015ecc'), 
             0.5), # el BE, wind input increased to 50%
            (('ecoinvent 3.7.1 conseq_fs', 'e5bf639cfd7f62017f0f72da8e0dc5f4'),
             ('ecoinvent 3.7.1 conseq_fs', '36f4649326c821b4e8dc8e005f83efd3'), 
             0.25), # el BE, nat gas input reduced to 25%
             (("fg_bw", "biowood production"),
              ('ecoinvent 3.7.1 conseq_fs', 'e5bf639cfd7f62017f0f72da8e0dc5f4'),
              10)] # biowood, el input, ten times higher

This is to show the coordinates (column number and row number) in the technology matrix and current values

In [50]:
for i in to_change:
    col = LCA.activity_dict[i[0]]
    row = LCA.activity_dict[i[1]]
    print(col, row, LCA.technosphere_matrix[row,col]) # shows current values

14498 4252 -0.2009371966123581
14498 16908 -0.557342529296875
17328 14498 -1.0


Using the coordinates of the activities and exchanges to be changed, **update the value**

In [51]:
for i in to_change:
    col = LCA.activity_dict[i[0]] # find index of tech matrix for the activity
    row = LCA.activity_dict[i[1]] # find index of tech matrix for the exchange
    LCA.technosphere_matrix[row,col] = -i[2] # substitute the value, need to change the sign!
    
    print(col, row, LCA.technosphere_matrix[row,col])

14498 4252 -0.5
14498 16908 -0.25
17328 14498 -10.0


Now perform calculations using the modified tech matrix

In [53]:
LCA.redo_lci() # uses the new tech matrix
LCA.lcia()
LCA.score/10 # because we have ten times higher input of electricity

0.14067654871132018

Ths score is diffent as before and lower impact as expected

### Using data from files (useful to organize scenarios and prepare things on beforehand)

Same but different, import the scenario from a separate file with the data, that you prepare on beforehand

In [54]:
scenario_data = pd.read_csv('scenario.csv', sep = ';') # import a csv unsing pandas, careful with formatting...
scenario_data

,activity_database,activity_code,exchange_database,exchange_code,value
0,ecoinvent 3.7.1 conseq_fs,e5bf639cfd7f62017f0f72da8e0dc5f4,ecoinvent 3.7.1 conseq_fs,89578c816af5949efef19da4d5015ecc,0.50
1,ecoinvent 3.7.1 conseq_fs,e5bf639cfd7f62017f0f72da8e0dc5f4,ecoinvent 3.7.1 conseq_fs,36f4649326c821b4e8dc8e005f83efd3,0.25
2,fg_bw,biowood production,ecoinvent 3.7.1 conseq_fs,e5bf639cfd7f62017f0f72da8e0dc5f4,10.00


Convert the dataframe in a list of tuples

In [57]:
to_change = []
for i in range(len(scenario_data.index)):
    change = ((scenario_data.iloc[i,0],scenario_data.iloc[i,1]),
              (scenario_data.iloc[i,2],scenario_data.iloc[i,3]),
              scenario_data.iloc[i,4])
    to_change.append(change)

In [58]:
to_change

[(('ecoinvent 3.7.1 conseq_fs', 'e5bf639cfd7f62017f0f72da8e0dc5f4'),
  ('ecoinvent 3.7.1 conseq_fs', '89578c816af5949efef19da4d5015ecc'),
  0.5),
 (('ecoinvent 3.7.1 conseq_fs', 'e5bf639cfd7f62017f0f72da8e0dc5f4'),
  ('ecoinvent 3.7.1 conseq_fs', '36f4649326c821b4e8dc8e005f83efd3'),
  0.25),
 (('fg_bw', 'biowood production'),
  ('ecoinvent 3.7.1 conseq_fs', 'e5bf639cfd7f62017f0f72da8e0dc5f4'),
  10.0)]

In [59]:
for i in to_change:
    col = LCA.activity_dict[i[0]] # find index of tech matrix for the activity
    row = LCA.activity_dict[i[1]] # find index of tech matrix for the exchange
    LCA.technosphere_matrix[row,col] = -i[2] # substitute the value
    #print(LCA.technosphere_matrix[row,col])

In [61]:
LCA.redo_lci() # uses the new tech matrix
LCA.lcia()
LCA.score/10

0.14067654871132018

If you have multiple scenarios, you can import each file with a different name and then **iterate the calculation across the scenarios**